In [ ]:
# !hugging_hub login
# !export HUGGINFACE_KEY_API=""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import textwrap
from typing import List

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def get_prompt_by_list(instruction: List[str], new_system_prompt=DEFAULT_SYSTEM_PROMPT):
  SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
  return [B_SYS+SYSTEM_PROMPT+instr+E_INST for instr in instruction]

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
# !pip install virtualenv
!pip install transformers
!pip install sentencepiece
!pip install langchain
!pip install sentence_transformers
!pip install bitsandbytes-cuda110
!pip install accelerate
!pip install bitsandbytes
# !pip3 install torch torchvision torchaudio
# !pip install packaging
# !pip install flash-attn --no-build-isolation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, BitsAndBytesConfig, TextStreamer, LlamaForCausalLM
import torch
import sys
from torch import cuda, bfloat16
import transformers
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.chains import LLMChain, create_extraction_chain
# sys.path.append("/content/drive/MyDrive/CoLab/Llama2/bin/python3.10/site-packages")

In [ ]:
# very important part, we quantilize Llama model by these codes accerlate
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# !nvidia-smi
# !sudo fuser -v /dev/nvidia0
# !sudo kill -9 406

In [ ]:
from transformers.pipelines.base import AutoConfig
import accelerate
import bitsandbytes

tokenizer=AutoTokenizer.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b")  # daryl149/llama-2-7b-chat-hf
model=LlamaForCausalLM.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b",quantization_config=quant_config,device_map="auto",)
tokenizer.pad_token = tokenizer.eos_token

# model=AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b", trust_remote_code=False, torch_dtype=torch.float16)
# config=AutoConfig.from_pretrained(
#     pretrained_model_name_or_path="/content/drive/MyDrive/CoLab/LLama7b",
#     top_p=0.67, top_k=15, repetition_penalty=1.2
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# load on text splitter and initialize text loading
loader=TextLoader(r"/content/drive/MyDrive/CoLab/Test_data/risk_abstract.txt")
text=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=4050, chunk_overlap=20)
docs=text_splitter.split_documents(text)

Do we need a Promopt template? Yes we need



In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

device=f"cuda: {cuda.current_device()}"
!nvidia-smi

Mon Oct 23 15:19:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    33W /  70W |   5015MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import asyncio
# Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]
# keyword and tags
def consutrct_instruction(words: str):
  instruction = f"""Help me to identify and summarize risks may raised in the given sentence "{words}".
    Please return risk may caused in the sentence and also pick risk type from given list:
    [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]
    If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'.

    Answer example:
    risks:  \n

    risk_type: \n

    Attention, each risk should be summarized in 4 words and summarize with suffix word 'risk'
    Attention, you can only output risk type with words from above list.
    Attention, only risks and risk_type should be presented.
    Attention, you can only predict the output in the above given criteria.
    """
  return instruction
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

prefix_llm=transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True, batch_size=4,
    task="text-generation",# temperature=0,
    top_p=0.15, top_k=15,
    max_new_tokens=1060, repetition_penalty=1.2, do_sample=True
)

In [ ]:
"""
a paradigm for direct huggingface transformer loading usage
what about agenerate?
"""
async def generate(text, modelv):
    prompt,number = get_prompt(consutrct_instruction(text)),text[:6]
    with torch.autocast('cuda', dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        print(len(inputs))
        outputs = modelv.generate(**inputs,
                                 max_new_tokens=256,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                top_p=0.15, top_k=15, repetition_penalty=1.2,do_sample=True
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs+"  "+str(number), number # , outputs

def generate_noasync(text, modelv):
    prompt = get_prompt(consutrct_instruction(text))
    with torch.autocast('cuda', dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        print(len(inputs))
        outputs = modelv.generate(**inputs,
                                 max_new_tokens=256,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                top_p=0.15, top_k=15, repetition_penalty=1.2,do_sample=True
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs # , outputs

In [ ]:
words=docs[0].page_content
context_pure=[get_prompt(consutrct_instruction(docs[i].page_content), system_prompt) for i in range(6)]
template = get_prompt(consutrct_instruction(words), system_prompt)
# print(template)
# res=generate(template)

In [ ]:
print(res)

In [ ]:
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor

# # Measure execution time of generate
# sync_start_time = time.time()
# generate_noasync(text, prefix_llm)
# sync_end_time = time.time()
# sync_execution_time = sync_end_time - sync_start_time
# print("Async generate execution time:", async_execution_time)
excutorpool=ThreadPoolExecutor(1)

async def test_generate(text):
    loop = asyncio.get_event_loop()
    start_time = time.time()
    await generate(text, prefix_llm)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time

async def main():
    start, lenth, interval=1154, 2000, 5
    text = [f"{val}"+docs[val].page_content for val in range(start, lenth)]
    output_list: List[str]=[]
    task, mints, maxts=[], 85, 10

    # Measure execution time of async generate
    async_start_time = time.time()
    for num in range(0, lenth, interval):
      round_time=time.time()
      for txt in text[num: num+interval]: task.append(asyncio.create_task(generate(txt, model)))
      output_list.extend(await asyncio.gather(*task, return_exceptions=True))
      task.clear()
      round_end=time.time()-round_time
      if round_end>maxts: maxts=round_end
      elif round_end<mints: mints=round_end

      if len(output_list)>50:
        with open("/content/drive/MyDrive/CoLab/Llama2/output2.txt", "a") as file:
          file.writelines(val[0]+"\n\n\n" for val in output_list)
        output_list.clear()

    async_end_time = time.time()
    async_execution_time = async_end_time - async_start_time

    print(f"{lenth-start} text, {round((lenth-start)/interval)} round async generate execution time:", async_execution_time)
    print(f"Max one round time: {maxts}, Min one round time: {mints}")
    return output_list

# Run the async function
output= await main()

with open("/content/drive/MyDrive/CoLab/Llama2/output2.txt", "a") as file:
  file.writelines(val[0]+"\n\n\n" for val in output)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


UnicodeEncodeError: ignored

In [ ]:
for val in output[:100]:
  print(val[1])


In [ ]:
print(len(output))
for val in range(6): print(output[val][0])

6
  Based on the provided sentence, I have identified several risks that may be present:
Risks:
* Uncertainty around the effectiveness of the proposed approach in mitigating both pathogen and DBP risks
* Potential for trade-offs between risk reduction measures for different contaminants
* Limited understanding of the impact of regulations on the cost and feasibility of implementing the recommended approach
* Possibility of unintended consequences arising from the implementation of the regulations

Risk Type: Environmental Science Risk  1154In
  Based on the provided sentence, here are some potential risks that could be identified:
Risks:
* Health risks due to poor air quality
* Environmental degradation due to unsustainable practices
* Social inequality and discrimination
* Economic instability due to over-reliance on a single industry
* Technological failures leading to infrastructure collapse

Risk Type: Environmental

Please note that these are just examples, and I cannot provide a 

In [ ]:
prefix_llm(context_pure, batch_size=6,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,)

In [ ]:
# model.config.eos_token_id
print(tokenizer.eos_token_id)

2


In [ ]:
with open("/content/drive/MyDrive/CoLab/Llama2/output.txt", "a") as file:
  file.writelines(val[0]+"\n\n\n" for val in output)